# ML model for Keyword Classification - Tech Notebook
This notebook introduces (1) how to explore, prepare and preprocess the datasets; (2) how to train and evaluate the ML model; and (3) how to use this trained ML model, for technical audiences.
## Problem Description
The AODN catalogue $C=\{M, K, P\}$ serves as a platform for storing datasets and their associated metadata. $M=\{m_1,m_2,\ldots, m_x\}$ is a set of metadata records which are used to describe the dataset in AODN catalogue $C$. $K=\{k_1, k_2, \ldots, k_y\}$ is a set of pre-defined keywords that are used to categorise dataset. In the catalogue $C = \{M, K\}$, a subset of metadata records, $M_t \subseteq M$, have not yet been categorised with keywords. For these records, $K_i = \emptyset $ for all $m_i \in M_t$. Given another subset of metadata records, $M_s \subseteq M$, where each record has already been categorised with keywords (i.e., $K_i \neq \emptyset $ for all $m_i \in M_s$). The research question is as follows:

How to design and develop a machine learning model, denoted as $MM_{keywords}$, that can automatically label the uncategorised metadata records $M_t$ using a predefined set of keywords $K$. Specifically, the model should be trained to learn a mapping rule $d_i \mapsto K_i$ based on the observed patterns from the sample set $M_s$, where each description $d_i$ of a metadata record $m_i \in M_s$ is associated with a set of keywords $K_i$. Once trained, the model should be able to apply this learned mapping to accurately categorise the records in $M_t$ by assigning their corresponding keywords based on the records' descriptions.

To simplify the task, we restrict the scope of keywords to those falling within the primary AODN vocabulary:
- AODN Instrument Vocabulary
- AODN Discovery Parameter Vocabulary
- AODN Platform Vocabulary

Only keywords $k_j \in K_i$ that are part of the listed AODN vocabularies will be considered. Any keyword not belonging to these vocabularies will be excluded from $K_i$ for all metadata records in the categorised metadata set $M_s$.

### Formal Definitions
- **Definition 1: A metadata record $m_i=(d_i, K_i), m_i \in M$** is a record describing a dataset. Specifically, $i$ is the unique identifier of the record. $d_i$ is a textual abstract that serves as the description of the dataset. $K_i \subseteq K$ is a subset of keywords used to label the dataset.
- **Definition 2: A abstract $d_i$** is a piece of textual information which is used to describe the dataset. The embedding $\mathbf{d_i}$ is a vector representation of the textual description $d_i$, calculated using the "bert-base-uncased" model. The embedding vector $\mathbf{d_i}$ for each abstract $d_i$ has an universal dimensionality, denoted as $dim=|\mathbf{d_i}|$. A feature matrix $\mathbf{X}$ of a shape $|M_s| \times dim$ aggregates the embeddings for the abstacts of all samples in $M_s$, where |M_s is the total number of metadata records.
- **Definition 3: A keyword $k_j$** is a predefined label used for catogarising datasets. Each metadata record $m_i$ is associated with a set of keywords $K_i \subseteq K$, while $K$ is the complete set of predefined keywords. The keywords $K_i$ for a metadata record $m_i$ is mathematiacally represented as a binary vector $y_i$ with a size of $|K|$. where each element indicates the presence or absence of a specific label. A value of 1 at position $j$ denotes the label $k_j \in K$ is present in the metadata record $m_i$, in this sence $k_j \in K_i$, while a value of 0 indicates its absence. A target matrix $\mathbf{Y}$ is a $|M_s| \times |K|$ binary matrix, where $|M_s|$ is the size of the metadata records set $M_s=\{m_1,m_2,\ldots, m_x\}$, and $|K|$ is the size of the keywords set $K=\{k_1, k_2, \ldots, k_y\}$. Each entry $ \mathbf{K}[i, j] $ is 1 if metadata record $ m_i $ is associated with keyword $ k_j $, and 0 otherwise.



In [1]:
# add module path for notebook to use
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\data_discovery_ai\\utils")
    sys.path.append(module_path+"\\data_discovery_ai\\model")
    sys.path.append(module_path+"\\data_discovery_ai\\common")

current_path = os.getcwd()

# import modules
import preprocessor
import keywordModel
import constants
import es_connector

c:\Users\yhu12\AppData\Local\miniforge3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


As shown in the [framework](data-discovery-ai-framework.drawio.png), three distinct but connected modules work cooperatively as the keyword classifier pipeline. This notebook will go through the functions in these modules to show how we preprocess data, train the ML model, and make predictions.
## Data Preprocessing
The data preprocessing module is used to prepare data for training and testing models. Key features include: getting raw data, preparing sample data, converting textual data to numeric representations, resampling, and preparing input and output matrices.
### Getting Raw Data
Raw data means the all metadata records $M$ stored in Elasticsearch. A elasticsearch configuration file `esManager.ini` is needed to be created in folder `data_discoverty_ai/common`, in which two fields are required: `end_point` and `api_key`. For more information, please refer to [README](../README.md#file-structure). We first fetch raw data from Elasticsearch.

In [2]:
# load Elasticsearch configuration
import configparser
from pathlib import Path

def load_es_config() -> configparser.ConfigParser:
    elasticsearch_config_file_path = f"../data_discovery_ai/common/{constants.ELASTICSEARCH_CONFIG}"
    esConfig = configparser.ConfigParser()
    esConfig.read(elasticsearch_config_file_path)
    return esConfig

In [3]:
# connect and query Elasticsearch
esConfig = load_es_config()
client = es_connector.connect_es(esConfig)
index = os.getenv("ES_INDEX_NAME", default=constants.ES_INDEX_NAME)
raw_data = es_connector.search_es(client=client, index=index, batch_size=constants.BATCH_SIZE, sleep_time=constants.SLEEP_TIME)

searching elasticsearch: 100%|██████████| 49/49 [04:20<00:00,  5.32s/it]


### Identify Samples
Sample set is a subset of the raw dataset. A sample set $M_s$ is a set of metadata records in which keywords contain particular AODN vocabus. We first identify samples from raw data, and then preprocess the sample set.

In [4]:
# get predefined vocabs
def load_keyword_config() -> configparser.ConfigParser:
    keyword_config_file_path = f"../data_discovery_ai/common/{constants.KEYWORD_CONFIG}"
    keywordConfig = configparser.ConfigParser()
    keywordConfig.read(keyword_config_file_path)
    return keywordConfig
keywordConfig = load_keyword_config()
vocabs = keywordConfig["preprocessor"]["vocabs"].split(", ")
vocabs

['AODN Instrument Vocabulary',
 'AODN Discovery Parameter Vocabulary',
 'AODN Platform Vocabulary']

The identified sample lables look like this format: 

In [5]:
# identify samples with predefined vocabs
identified_sampleSet = preprocessor.identify_sample(raw_data, vocabs)
identified_sampleSet.iloc[0]["keywords"]

[{'concepts': [{'id': 'Oceans | Ocean Temperature | Water Temperature'},
   {'id': 'Oceans | Ocean Optics | Photosynthetically Active Radiation'},
   {'id': 'Oceans | Ocean Optics | Turbidity'},
   {'id': 'Atmosphere | Precipitation | Rain'},
   {'id': 'Oceans | Ocean Chemistry | Chlorophyll'},
   {'id': 'Oceans | Salinity/density | Salinity'}],
  'scheme': 'theme',
  'description': 'GCMD',
  'title': 'NASA/Global Change Master Directory Earth Science Keywords Version 5.3.8'},
 {'concepts': [{'id': 'Buoys | Moored Buoys'},
   {'id': 'Fluorometers'},
   {'id': 'CTD (Conductivity-Temperature-Depth Profilers)'}],
  'scheme': '',
  'description': 'MCP',
  'title': 'Marine Community Profile of ISO19115 v1.4 Collection Methods Vocabulary (Annex C.1.3)'},
 {'concepts': [{'id': 'IMOS Platform | NRSDAR | Darwin National Reference Station Mooring'},
   {'id': 'IMOS Platform | NRSYON | Yongala National Reference Station Mooring'},
   {'id': 'Wind speed'},
   {'id': 'Wind direction'},
   {'id': 'B

The keywords is in a nested json format, we need to flattern them, and remove keywords which are not in the target vocabularies.

In [6]:
preprocessed_SampleSet = preprocessor.sample_preprocessor(identified_sampleSet, vocabs)
preprocessed_SampleSet

,id,title,description,keywords
12,006bb7dc-860b-4b89-bf4c-6bd930bd35b7,IMOS - ANMN National Reference Stations - Darw...,This collection includes observations transmit...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v..."
16,0094682a-e438-41e8-a39b-19cf2093025d,Thursday Island Wind From 08 Feb 2012,This data set was collected by weather sensors...,[{'vocab_type': 'AODN Discovery Parameter Voca...
20,00a64d43-86a8-4f2b-89e6-40f1abf288f6,Cumulative Pressures on the Distinctive Values...,A report was developed by the Western Australi...,[]
28,00fee0c8-6203-4271-8d46-f36c075fa6cf,IMOS SOOP Underway Data from AIMS Vessel RV So...,'Ships of Opportunity' (SOOP) is a facility of...,[{'vocab_type': 'AODN Discovery Parameter Voca...
34,0145df96-3847-474b-8b63-a66f0e03ff54,Statewide Marine Habitat Map 2023,The Statewide Marine Habitat Map 2023 was deve...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v..."
...,...,...,...,...
4891,6a0ba674-f7c0-4590-baba-2242c3dc265b,Temperature Data for the GBR Region (NESP TWQ ...,This collection of temperature logger data has...,[{'vocab_type': 'AODN Discovery Parameter Voca...
4900,6a379324-692d-436c-9b7a-afe29f4dd43b,IMOS SOOP Underway Data from AIMS Vessel RV Ca...,'Ships of Opportunity' (SOOP) is a facility of...,[{'vocab_type': 'AODN Discovery Parameter Voca...
4903,6a478253-37a8-4582-af91-165ad7d9ff9c,Hardy Reef Air Pressure From 14 Jun 1989,This data set was collected by weather sensors...,[{'vocab_type': 'AODN Discovery Parameter Voca...
4907,6a56db3f-d1b2-438d-98c6-bd7dd540a4d5,Reef Life Survey (RLS): Cryptic Fish,[This data has been superseded by a synthesise...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v..."


Clean the sample set, for instance, row at index `20` has an empty keyword filed like `[]`

In [7]:
filtered_sampleSet = preprocessed_SampleSet[preprocessed_SampleSet["keywords"].apply(lambda x: x != [])]
filtered_sampleSet

,id,title,description,keywords
12,006bb7dc-860b-4b89-bf4c-6bd930bd35b7,IMOS - ANMN National Reference Stations - Darw...,This collection includes observations transmit...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v..."
16,0094682a-e438-41e8-a39b-19cf2093025d,Thursday Island Wind From 08 Feb 2012,This data set was collected by weather sensors...,[{'vocab_type': 'AODN Discovery Parameter Voca...
28,00fee0c8-6203-4271-8d46-f36c075fa6cf,IMOS SOOP Underway Data from AIMS Vessel RV So...,'Ships of Opportunity' (SOOP) is a facility of...,[{'vocab_type': 'AODN Discovery Parameter Voca...
34,0145df96-3847-474b-8b63-a66f0e03ff54,Statewide Marine Habitat Map 2023,The Statewide Marine Habitat Map 2023 was deve...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v..."
37,0155375c-8070-4662-9c93-b593ee4891b0,Davies Reef Water Temperature From 18 Oct 1991,The 'Wireless Sensor Networks Facility' (forme...,[{'vocab_type': 'AODN Discovery Parameter Voca...
...,...,...,...,...
4891,6a0ba674-f7c0-4590-baba-2242c3dc265b,Temperature Data for the GBR Region (NESP TWQ ...,This collection of temperature logger data has...,[{'vocab_type': 'AODN Discovery Parameter Voca...
4900,6a379324-692d-436c-9b7a-afe29f4dd43b,IMOS SOOP Underway Data from AIMS Vessel RV Ca...,'Ships of Opportunity' (SOOP) is a facility of...,[{'vocab_type': 'AODN Discovery Parameter Voca...
4903,6a478253-37a8-4582-af91-165ad7d9ff9c,Hardy Reef Air Pressure From 14 Jun 1989,This data set was collected by weather sensors...,[{'vocab_type': 'AODN Discovery Parameter Voca...
4907,6a56db3f-d1b2-438d-98c6-bd7dd540a4d5,Reef Life Survey (RLS): Cryptic Fish,[This data has been superseded by a synthesise...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v..."


Then we calculate embeddings for the title and description field, which is used as the input feature matrix.

In [8]:
finalSampleSet = preprocessor.calculate_embedding(filtered_sampleSet)

c:\Users\yhu12\OneDrive - University of Tasmania\IMOS\DataDiscovery\data-discovery-ai\data_discovery_ai\utils\preprocessor.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds["information"] = ds["title"] + ": " + ds["description"]
100%|██████████| 647/647 [14:58<00:00,  1.39s/it]
c:\Users\yhu12\OneDrive - University of Tasmania\IMOS\DataDiscovery\data-discovery-ai\data_discovery_ai\utils\preprocessor.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds["embedding"] = ds["information"].progress_appl

In [9]:
finalSampleSet

,id,title,description,keywords,information,embedding
12,006bb7dc-860b-4b89-bf4c-6bd930bd35b7,IMOS - ANMN National Reference Stations - Darw...,This collection includes observations transmit...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v...",IMOS - ANMN National Reference Stations - Darw...,"[-0.80773824, 0.19351113, 0.02368839, -0.34996..."
16,0094682a-e438-41e8-a39b-19cf2093025d,Thursday Island Wind From 08 Feb 2012,This data set was collected by weather sensors...,[{'vocab_type': 'AODN Discovery Parameter Voca...,Thursday Island Wind From 08 Feb 2012: This da...,"[-0.6913842, -0.48216444, 0.51189363, 0.038675..."
28,00fee0c8-6203-4271-8d46-f36c075fa6cf,IMOS SOOP Underway Data from AIMS Vessel RV So...,'Ships of Opportunity' (SOOP) is a facility of...,[{'vocab_type': 'AODN Discovery Parameter Voca...,IMOS SOOP Underway Data from AIMS Vessel RV So...,"[-0.7061849, -0.5095051, 0.4899681, -0.0899401..."
34,0145df96-3847-474b-8b63-a66f0e03ff54,Statewide Marine Habitat Map 2023,The Statewide Marine Habitat Map 2023 was deve...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v...",Statewide Marine Habitat Map 2023: The Statewi...,"[-1.2934202, -0.40619895, -0.014477678, -0.177..."
37,0155375c-8070-4662-9c93-b593ee4891b0,Davies Reef Water Temperature From 18 Oct 1991,The 'Wireless Sensor Networks Facility' (forme...,[{'vocab_type': 'AODN Discovery Parameter Voca...,Davies Reef Water Temperature From 18 Oct 1991...,"[-0.83987963, -0.3394436, 0.35684508, -0.11953..."
...,...,...,...,...,...,...
4891,6a0ba674-f7c0-4590-baba-2242c3dc265b,Temperature Data for the GBR Region (NESP TWQ ...,This collection of temperature logger data has...,[{'vocab_type': 'AODN Discovery Parameter Voca...,Temperature Data for the GBR Region (NESP TWQ ...,"[-1.0290724, -0.3438952, 0.38878796, -0.217064..."
4900,6a379324-692d-436c-9b7a-afe29f4dd43b,IMOS SOOP Underway Data from AIMS Vessel RV Ca...,'Ships of Opportunity' (SOOP) is a facility of...,[{'vocab_type': 'AODN Discovery Parameter Voca...,IMOS SOOP Underway Data from AIMS Vessel RV Ca...,"[-0.68497896, -0.49998918, 0.49762017, -0.1424..."
4903,6a478253-37a8-4582-af91-165ad7d9ff9c,Hardy Reef Air Pressure From 14 Jun 1989,This data set was collected by weather sensors...,[{'vocab_type': 'AODN Discovery Parameter Voca...,Hardy Reef Air Pressure From 14 Jun 1989: This...,"[-0.78799915, -0.25346056, 0.54476476, -0.3033..."
4907,6a56db3f-d1b2-438d-98c6-bd7dd540a4d5,Reef Life Survey (RLS): Cryptic Fish,[This data has been superseded by a synthesise...,"[{'vocab_type': 'AODN Platform Vocabulary', 'v...",Reef Life Survey (RLS): Cryptic Fish: [This da...,"[-0.8747705, -0.38033172, -0.42333993, -0.0655..."


### Prepare Train and Test Sets
We now have the sample set with extra embedding information. We are going to split the sample set into train and test sets by preparing input feature matrix $X$ and output target matrix $Y$. The input feature matrix X is based on the embedding column, and the output Y is the mathmatic representation of the keyword column.

In [10]:
X, Y, Y_df, labelMap = preprocessor.prepare_X_Y(finalSampleSet)

c:\Users\yhu12\OneDrive - University of Tasmania\IMOS\DataDiscovery\data-discovery-ai\data_discovery_ai\utils\preprocessor.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds["keywordsMap"] = ds["keywords"].apply(


We have prepared the input feature matrix `X` and the output target matrix `Y`. Additionally, we have `Y_df`, which includes column names for the `Y` matrix, and `labelMap`, which represents the keyword set of predefined keywords. In `labelMap`, the key is an encoded number corresponding to a column name in `Y_df`, and the value is a Concept object. We can review the details of a Concept object by its `to_json()` function.

In [11]:
Y_df

,0,1,2,3,4,5,6,7,8,9,...,329,330,331,332,333,334,335,336,337,338
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
labelMap.get(0).to_json()

{'vocab_type': 'AODN Discovery Parameter Vocabulary',
 'value': 'wet weight biomass per unit volume of the water body',
 'url': 'http://vocab.aodn.org.au/def/discovery_parameter/entity/571'}

In [15]:
rare_label_index = preprocessor.identify_rare_labels(Y_df, constants.RARE_LABEL_THRESHOLD, list(labelMap.keys()))
len(rare_label_index)

199

We found that among 339 unique keywords, there are 199 keywords appears less than the `RARE_LABEL_THRESHOLD`. So we firstly duplicate records which have these rare labels with a customised resamplying.

In [16]:
X_oversampled, Y_oversampled = preprocessor.resampling(
            X_train=X, Y_train=Y, strategy="custom", rare_keyword_index=rare_label_index
        )

 ======== After Resampling ========
Total samples: 1677
Dimension: 768
No. of labels: 339
X resampled set size: 1677
Y resampled set size: 1677


Now the sample size is increased from 647 to 1677 so that the records of rare labels are manually increased. We can now split the sample set to train and test sets follows a 80%-20% split.

In [18]:
dim, n_labels, X_train, Y_train, X_test, Y_test = (
            preprocessor.prepare_train_test(X_oversampled, Y_oversampled, keywordConfig)
        )

INFO:preprocessor:Total samples: 1677
INFO:preprocessor:Dimension: 768
INFO:preprocessor:No. of labels: 339
INFO:preprocessor:Train set size: 1340 (79.90%)
INFO:preprocessor:Test set size: 337 (20.10%)


Next, we perform oversampling only on the training set, as we want to avoid introducing training samples into the test set. This ensures the model does not encounter training data during testing.

In [19]:
X_train_oversampled, Y_train_oversampled = preprocessor.resampling(
            X_train=X_train, Y_train=Y_train, strategy="ROS", rare_keyword_index=None
        )

 ======== After Resampling ========
Total samples: 19596
Dimension: 768
No. of labels: 339
X resampled set size: 19596
Y resampled set size: 19596


Then, we calculate the class weight, so that we can apply in model training by assigning majority classes lower weight, and minority classes higher weight.

In [21]:
label_weight_dict = keywordModel.get_class_weights(Y_train)

Now, we have prepared all the data we need for training a keyword classification model. Let's move on to the next stage.

## Training and Evaluation of Model
A model name is required for training a model. As mentioned in [README.md](../README.md), available options are: `development`,`experimental`, `staging`, `production`, `benchmark`. 

In [23]:
model_name = "experimental"

In [24]:
trained_model, history, model_name = keywordModel.keyword_model(
            model_name=model_name,
            X_train=X_train,
            Y_train=Y_train,
            X_test=X_test,
            Y_test=Y_test,
            class_weight=label_weight_dict,
            dim=dim,
            n_labels=n_labels,
            params=keywordConfig,
        )

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 339)            │        43,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,163 (555.32 KB)

 Trainable params: 142,163 (555.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0193 - loss: 0.0016 - precision: 0.0344 - recall: 0.3441 - val_accuracy: 0.0000e+00 - val_loss: 0.0187 - val_precision: 0.4088 - val_recall: 0.0869 - learning_rate: 0.0010
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0867 - loss: 5.4025e-04 - precision: 0.3339 - recall: 0.2044 - val_accuracy: 0.0000e+00 - val_loss: 0.0148 - val_precision: 0.7978 - val_recall: 0.2211 - learning_rate: 0.0010
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1128 - loss: 4.3541e-04 - precision: 0.6218 - recall: 0.3535 - val_accuracy: 0.0000e+00 - val_loss: 0.0119 - val_precision: 0.9097 - val_recall: 0.5129 - learning_rate: 0.0010
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1170 - loss: 3.2388e-04 - precision: 0.7449 - recall: 0.5092 - val_accuracy: 0.0634 - val_loss: 0.0095 - val_precision: 0.9574 - val_recall: 0.6259 - learning_rate: 0.0010
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

Then, we evaluate the trained model.

In [25]:
confidence = keywordConfig.getfloat("keywordModel", "confidence")
top_N = keywordConfig.getint("keywordModel", "top_N")
predicted_labels = keywordModel.prediction(
    X_test, trained_model, confidence, top_N
)
eval = keywordModel.evaluation(
    Y_test=Y_test, predictions=predicted_labels
)
eval

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


{'precision': '0.9074',
 'recall': '0.9549',
 'f1': '0.9305',
 'hammingloss': '0.0041',
 'Jaccard Index': '0.8159',
 'accuracy': '0.7122'}

We had a result of 90% precision, 95% recall, and 93% F1 score. Which is not bad. But we can still try different hypermeters to improve model performance. Please refer to [README.md](../README.md) to see hypermeter descriptions. To adjust model hypermeters, please go to file `data_discovery_ai\common\keyword_classification_parameters.ini` to try different values.

## Make Prediction

Now we have the trained model, let's use this model to make prediction. Let's assume we have a item entitled: *"Corals and coral communities of Lord Howe Island, Australia"* with an abstract *"Ecological and taxonomic surveys of hermatypic scleractinian corals were carried out at approximately 100 sites around Lord Howe Island. Sixty-six of these sites were located on reefs in the lagoon, which extends for two-thirds of the length of the island on the western side. Each survey site consisted of a section of reef surface, which appeared to be topographically and faunistically homogeneous. The dimensions of the sites surveyed were generally of the order of 20m by 20m. Where possible, sites were arranged contiguously along a band up the reef slope and across the flat. The cover of each species was graded on a five-point scale of percentage relative cover. Other site attributes recorded were depth (minimum and maximum corrected to datum), slope (estimated), substrate type, total estimated cover of soft coral and algae (macroscopic and encrusting coralline). Coral data from the lagoon and its reef (66 sites) were used to define a small number of site groups which characterize most of this area.Throughout the survey, corals of taxonomic interest or difficulty were collected, and an extensive photographic record was made to augment survey data. A collection of the full range of form of all coral species was made during the survey and an identified reference series was deposited in the Australian Museum.In addition, less detailed descriptive data pertaining to coral communities and topography were recorded on 12 reconnaissance transects, the authors recording changes seen while being towed behind a boat.
 The purpose of this study was to describe the corals of Lord Howe Island (the southernmost Indo-Pacific reef) at species and community level using methods that would allow differentiation of community types and allow comparisons with coral communities in other geographic locations."* that is unlabelled.

In [27]:
item_title = "Corals and coral communities of Lord Howe Island, Australia"
item_abstract = """Ecological and taxonomic surveys of hermatypic scleractinian corals were carried out at approximately 100 sites around Lord Howe Island. Sixty-six of these sites were located on reefs in the lagoon, which extends for two-thirds of the length of the island on the western side. Each survey site consisted of a section of reef surface, which appeared to be topographically and faunistically homogeneous. The dimensions of the sites surveyed were generally of the order of 20m by 20m. Where possible, sites were arranged contiguously along a band up the reef slope and across the flat. The cover of each species was graded on a five-point scale of percentage relative cover. Other site attributes recorded were depth (minimum and maximum corrected to datum), slope (estimated), substrate type, total estimated cover of soft coral and algae (macroscopic and encrusting coralline). Coral data from the lagoon and its reef (66 sites) were used to define a small number of site groups which characterize most of this area.Throughout the survey, corals of taxonomic interest or difficulty were collected, and an extensive photographic record was made to augment survey data. A collection of the full range of form of all coral species was made during the survey and an identified reference series was deposited in the Australian Museum.In addition, less detailed descriptive data pertaining to coral communities and topography were recorded on 12 reconnaissance transects, the authors recording changes seen while being towed behind a boat.
 The purpose of this study was to describe the corals of Lord Howe Island (the southernmost Indo-Pacific reef) at species and community level using methods that would allow differentiation of community types and allow comparisons with coral communities in other geographic locations."""
description = f"{item_title}: {item_abstract}"

We first prepare input feature matrix X, which is the embedding of this description.

In [28]:
description_embedding = preprocessor.get_description_embedding(description)
dimension = description_embedding.shape[0]
target_X = description_embedding.reshape(1, dimension)
target_X

array([[-7.76477098e-01, -3.33351672e-01, -7.07494497e-01,
         2.77853608e-02,  1.66572064e-01,  9.25325900e-02,
         3.53269756e-01,  3.13511342e-01,  3.27710301e-01,
        -2.67760873e-01, -6.13279700e-01, -2.74785250e-01,
        -2.21668690e-01,  7.12784171e-01,  8.41400586e-03,
         5.41278899e-01,  1.35883421e-01,  1.88588262e-01,
         2.41953388e-01,  3.85780513e-01, -4.87768143e-01,
        -3.06539148e-01,  2.47475669e-01,  3.70917350e-01,
        -4.72536981e-01, -3.74548137e-01, -1.55186579e-01,
         2.62748361e-01, -8.84262845e-02, -3.37665975e-01,
        -1.81324035e-01,  5.80801129e-01, -1.15090990e+00,
        -1.01637912e+00, -4.25794758e-02,  3.44661593e-01,
        -6.89785540e-01,  1.17133960e-01,  1.27914354e-01,
        -2.11383685e-01, -4.57495719e-01,  3.49888206e-01,
         3.89928788e-01, -2.88182795e-01,  2.89381206e-01,
         4.38351631e-01, -4.48247337e+00,  1.56454593e-01,
        -2.23044991e-01,  1.47479400e-02,  4.61551875e-0

Then we use the trained model and X to make prediction

In [30]:
target_predicted_labels = keywordModel.prediction(
        target_X,
        trained_model,
        keywordConfig.getfloat("keywordModel", "confidence"),
        keywordConfig.getint("keywordModel", "top_N"),
    )
target_predicted_labels

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

It's in a binary format, but means that at which index the values are 1, the keywords at these index have a higher probability to be appeared in the item. So, we convert this binary array to readable format.

In [ ]:
prediction = keywordModel.get_predicted_keywords(target_predicted_labels, labelMap)
prediction